In [1]:
!pip install pymongo

In [2]:
import pymongo

# Create a MongoDB client
client = pymongo.MongoClient()

# Select the database and collections to be used
db = client["Flight_Delay_Analysis"]
collections = [db[str(year)] for year in range(1987, 1999)]


print(collections)

[Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Flight_Delay_Analysis'), '1987'), Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Flight_Delay_Analysis'), '1988'), Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Flight_Delay_Analysis'), '1989'), Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Flight_Delay_Analysis'), '1990'), Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Flight_Delay_Analysis'), '1991'), Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Flight_Delay_Analysis'), '1992'), Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Flight_Delay_Analy

In [3]:
# Specify the airports to filter
airports = ["IAD", "ORD", "LAS", "BWI", "SLC", "BNA", "JFK", "SEA", "CVG", "BOS", "HOU", "DTW", "DEN", "ORL", "PIT", "MIA", "LAX",'SYR',"LGA"]

# Create a new collection to store the aggregated data
agg_col = db["aggregated_columns"]

# Loop over the collections and perform the aggregation
for collection in collections:
    pipeline = [
        {
            "$match": {
                "$or": [
                    {"Origin": {"$in": airports}},
                    {"Dest": {"$in": airports}}
                ]
            }
        },
        {
            "$project": {
                "_id": 1,
                "Year": 1,
                "Month": 1,
                "DayofMonth": 1,
                "DayOfWeek": 1,
                "DepTime": 1,
                "CRSDepTime": 1,
                "ArrTime": 1,
                "CRSArrTime": 1,
                "UniqueCarrier": 1,
                "FlightNum": 1,
                "TailNum": 1,
                "ActualElapsedTime": 1,
                "CRSElapsedTime": 1,
                "AirTime": 1,
                "ArrDelay": 1,
                "DepDelay": 1,
                "Origin": 1,
                "Dest": 1,
                "Distance": 1,
                "TaxiIn": 1,
                "TaxiOut": 1,
                "Cancelled": 1,
                "Diverted": 1,
                "CarrierDelay": 1,
                "WeatherDelay": 1,
                "NASDelay": 1,
                "SecurityDelay": 1,
                "LateAircraftDelay": 1
            }
        },
        {
            "$out": "temp"
        }
    ]
    collection.aggregate(pipeline)
    agg_col.insert_many(db["temp"].find())
    db["temp"].drop()

